In [1]:
import os

from dotenv import load_dotenv

load_dotenv()

True

# Tool Use


First of all, let's get the available tools from the MCP server.


In [43]:
from fastmcp.client import Client

mcp_client = Client("http://127.0.0.1:9090/mcp")

async with mcp_client:
    mcp_tools = await mcp_client.list_tools()

In [44]:
mcp_tools

[Tool(name='process_video', description='Process a video file and prepare it for searching.', inputSchema={'properties': {'video_path': {'title': 'Video Path', 'type': 'string'}}, 'required': ['video_path'], 'type': 'object'}, annotations=None),
 Tool(name='get_video_clip_from_user_query', description='Use this tool to get a video clip from a video file based on a user query or question.', inputSchema={'properties': {'video_name': {'title': 'Video Name', 'type': 'string'}, 'user_query': {'title': 'User Query', 'type': 'string'}}, 'required': ['video_name', 'user_query'], 'type': 'object'}, annotations=None),
 Tool(name='get_video_clip_from_image', description='Use this tool to get a video clip from a video file based on a user image.', inputSchema={'$defs': {'Base64Image': {'properties': {'image': {'description': 'Base64 encoded image string', 'title': 'Image', 'type': 'string'}}, 'required': ['image'], 'title': 'Base64Image', 'type': 'object'}}, 'properties': {'video_name': {'title': 

Now that we have the tools, it's time to translate them into the format that the Groq API expects.


In [45]:
from mcp.types import Tool


def transform_schema_to_parameters(schema: dict) -> dict:
    properties = {}

    for field_name, field_info in schema["properties"].items():
        properties[field_name] = {
            "type": field_info["type"],
            "description": field_info["title"],
        }
        if "default" in field_info:
            properties[field_name]["default"] = field_info["default"]

    return {
        "type": "object",
        "properties": properties,
        "required": schema.get("required"),
    }


def transform_tool_definition(tool: Tool) -> dict:
    return {
        "type": "function",
        "function": {
            "name": tool.name,
            "description": tool.description,
            "parameters": transform_schema_to_parameters(tool.inputSchema),
        },
    }

In [46]:
tools = [
    transform_tool_definition(mcp_tool)
    for mcp_tool in mcp_tools
    if mcp_tool.name != "get_video_clip_from_image"
]

In [47]:
tools

[{'type': 'function',
  'function': {'name': 'process_video',
   'description': 'Process a video file and prepare it for searching.',
   'parameters': {'type': 'object',
    'properties': {'video_path': {'type': 'string',
      'description': 'Video Path'}},
    'required': ['video_path']}}},
 {'type': 'function',
  'function': {'name': 'get_video_clip_from_user_query',
   'description': 'Use this tool to get a video clip from a video file based on a user query or question.',
   'parameters': {'type': 'object',
    'properties': {'video_name': {'type': 'string',
      'description': 'Video Name'},
     'user_query': {'type': 'string', 'description': 'User Query'}},
    'required': ['video_name', 'user_query']}}},
 {'type': 'function',
  'function': {'name': 'ask_question_about_video',
   'description': 'Use this tool to get an answer to a question about the video.',
   'parameters': {'type': 'object',
    'properties': {'video_name': {'type': 'string',
      'description': 'Video Name'

In [48]:
from groq import Groq

MODEL = "llama-3.3-70b-versatile"
groq_client = Groq(api_key=os.environ.get("GROQ_API_KEY"))


In [49]:
messages = [
    {"role": "system", "content": "You are Kubrick, a video processing assistant."},
    {
        "role": "user",
        "content": "Hey! What's your name?",
    },
]

response = groq_client.chat.completions.create(
    model=MODEL,
    messages=messages,
    stream=False,
    tools=tools,
    tool_choice="auto",
    max_completion_tokens=4096,
)

In [50]:
response.choices[0].message.content

"I'm Kubrick, nice to meet you. I'm a video processing assistant, here to help with any video-related tasks you might have. How can I assist you today?"

In [68]:
messages = [
    {
        "role": "system",
        "content": "You are Kubrick, a video processing assistant. The current video name is: ./videos/pass_the_butter_rick_and_morty.mp4",
    },
    {
        "role": "user",
        "content": "I want to get the clip of the video where Rick says 'Pass the butter', please",
    },
]

response = groq_client.chat.completions.create(
    model=MODEL,
    messages=messages,
    stream=False,
    tools=tools,
    tool_choice="auto",
    max_completion_tokens=4096,
)

In [69]:
response.choices[0].message.content

In [70]:
tool_calls = response.choices[0].message.tool_calls

In [71]:
tool_calls[0].function.name

'get_video_clip_from_user_query'

In [72]:
import json

inputs = json.loads(tool_calls[0].function.arguments)

In [73]:
inputs

{'user_query': 'Rick says Pass the butter',
 'video_name': './videos/pass_the_butter_rick_and_morty.mp4'}

In [74]:
async with mcp_client:
    result = await mcp_client.call_tool("get_video_clip_from_user_query", inputs)
    print(result)

[TextContent(type='text', text='{\n  "filename": "./videos/4c60b94d-9276-4d33-b881-3d652f583512.mp4"\n}', annotations=None)]


In [76]:
import json

json.loads(result[0].text)

{'filename': './videos/4c60b94d-9276-4d33-b881-3d652f583512.mp4'}

In [22]:
response_message = response.choices[0].message
tool_calls = response_message.tool_calls

In [23]:
tool_calls